# 4c. Get traffic sign features

In [ ]:
# Standard library and path imports
import set_path
import requests

# Third-party library imports
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame

import folium

# Local or project-specific imports
import plot_utils
import settings as st

if st.my_run == "azure":
    import config_azure as cf
elif st.my_run == "local":
    import config as cf

## Get traffic sign data

In [ ]:
response = requests.get('https://api.data.amsterdam.nl/dcatd/files/e85bcf2124fb4437b1bc6eb75dfc3abf_objectstore_eu/dcatd/0ebf0327d3124d848ada6611b71c3cd0')
if response.status_code == 200:
    with open(cf.signs_file, 'wb') as file:
        file.write(response.content)
    print(f"Downloaded traffic sign data as {cf.signs_file}")
else:
    print("Failed to retrieve traffic sign data")

In [ ]:
df_signs_all = pd.read_csv(cf.signs_file)
df_signs_all = GeoDataFrame(df_signs_all, geometry=gpd.points_from_xy(df_signs_all['x-coordinaat'], 
                                                                      df_signs_all['y-coordinaat']), crs=st.CRS)
df_signs_all = df_signs_all.drop(['x-coordinaat', 'y-coordinaat'], axis=1)

### Select crosswalk traffic signs

In [ ]:
#L02(F): Voetgangers oversteekplaats / zebrapad (blauw)
#J22(F): Voetgangersoversteekplaats met een zebrapad (vooraanduiding)  (rood)
#J23(F): U nadert een niet gemarkeerde voetgangers oversteekplaats

df_signs = df_signs_all[df_signs_all['rvv-modelnummer'].isin(['J22', 'J22F',
                                                              'L02', 'L02F', 'L02OB15F'])].reset_index(drop=True)
df_signs = df_signs.dropna(axis=1, how='all')  # remove columns without values

## Get mask of area

In [ ]:
# Import areas
df_areas = gpd.read_file(cf.output_pilot_area)
polygon = df_areas.to_crs(crs=st.CRS).unary_union

## Extract traffic sign information in polygon of area

In [ ]:
tl_data_in_area_mask = df_signs['geometry'].apply(lambda geom: polygon.contains(geom))
tl_data_in_area = df_signs.loc[tl_data_in_area_mask]

## Inspect data

In [ ]:
if tl_data_in_area.empty:
    print('No sidewalk data available in area')

tl_data_in_area

## Store

In [ ]:
# Save traffic sign crossing features as geopackage
tl_data_in_area.to_file(cf.output_traffic_sign_crossing_features, driver='GPKG')

## Visualize features

In [ ]:
# set True for satellite background, False for standard background
satellite = False

# Create Folium map
map = folium.Map(
    location=[52.350547922223434, 4.794019242371844], tiles=plot_utils.generate_map_params(satellite=satellite),
    min_zoom=10, max_zoom=25, zoom_start=17,
    zoom_control=True, control_scale=True, control=False
    )

# Add traffic sign features
feature_names = tl_data_in_area.columns.tolist()
feature_names.remove('geometry')
tooltip = plot_utils.gen_tooltip(feature_names, feature_names)
geo_j = folium.GeoJson(tl_data_in_area, tooltip=tooltip, style_function=lambda x: {"color": "orange", "weight": 10}).add_to(map)

map